In [2]:
# HuggingFace-transformers
from transformers import BertTokenizer, GPT2LMHeadModel, TextGenerationPipeline

# 实例化tokenizer和model
tokenizer = BertTokenizer.from_pretrained("uer/gpt2-distil-chinese-cluecorpussmall")
model = GPT2LMHeadModel.from_pretrained("uer/gpt2-distil-chinese-cluecorpussmall")

text_generator = TextGenerationPipeline(model, tokenizer)
text_generator("中 国 是 一 个 伟 大 的 国 家 吗 ？", max_length=200, do_sample=True)

c:\Users\ethan\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ethan\miniconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\ethan\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ethan\.cache\huggingface\hub\models--uer--gpt2-distil-chinese-cluecorpussmall. Caching files will still work but in a

[{'generated_text': '中 国 是 一 个 伟 大 的 国 家 吗 ？ 看 到 知 友 质 疑 有 评 论 说 我 错 了 ， 我 说 是 ， 我 又 是 来 自 于 父 国 的 反 华 势 力 。 我 的 祖 国 是 我 的 国 。 我 没 有 歧 视 祖 国 ， 我 说 国 也 是 我 的 国 。 但 是 我 可 以 反 驳 我 的 答 案 。 我 不 能 因 为 祖 国 只 是 一 个 国 家 就 否 定 祖 国 是 对 我 自 己 的 错 ？ 为 什 么 中 国 会 被 我 这 个 回 答 误 导 了 ？ 不 能 因 为 祖 国 只 是 一 个 国 家 。 祖 国 只 是 一 个 国 家 吗 ？ 这 样 的 问 题 ， 我 没 法 回 答 。 你 想 了 想 ， 如 果 你 觉 得 很 好 的 话 ， 请 转 告 给 我 的 朋 友 。 我 只 想 你 的 答 案'}]

In [ ]:
from datasets import Dataset  # 数据封装 (将文本数据封装为可以分批次拿来训练的张量)
from transformers import DataCollatorForLanguageModeling  # 数据整理
from transformers import BertTokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

# 实例化tokenizer和model
tokenizer = BertTokenizer.from_pretrained("uer/gpt2-distil-chinese-cluecorpussmall")
model = GPT2LMHeadModel.from_pretrained("uer/gpt2-distil-chinese-cluecorpussmall")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

max_length = 300

def split_text_into_chunks(text, max_len=512):
    chunks = []
    current_chunk = ""
    paragraphs = text.split("\n")
    paragraphs = [p.strip() for p in paragraphs if p.strip()]
    
    for paragraph in paragraphs:        
        tokens = tokenizer.tokenize(paragraph)
        if len(tokens) > max_len:
            tokens = tokens[:max_len]
            paragraph = tokenizer.convert_tokens_to_string(tokens)

        combined_chunk = current_chunk + paragraph + "  "
        combined_tokens = tokenizer.tokenize(combined_chunk)
        
        if len(combined_tokens) <= max_len:
            current_chunk = combined_chunk
        else:
            chunks.append(current_chunk.strip())
            current_chunk = paragraph + "  "
    
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks
        

sentences = []
data_path = r'./data/wiki_zh_2019/wiki_00'
import json
with open(data_path, "r", encoding="utf-8") as f:
    data = json.load(f)
    for item in data:
        text = item["text"]
        sentences.extend(split_text_into_chunks(text, max_len=max_length))

print(sentences[:3])

In [ ]:
# 使用tokenizer处理文本
encoding = tokenizer(sentences,
                     max_length=max_length,
                     padding='max_length',
                     truncation=True,
                     return_tensors='pt')
# return_tensors 参数还可以设置为以下值：
# 'tf' - 返回 TensorFlow 张量
# 'np' - 返回 NumPy 数组
# None - 返回 Python 列表（默认值）


data_dict = {
    'input_ids': encoding['input_ids'],  # 输入数据的 token 序列，通常通过分词器生成
    'attention_mask': encoding['attention_mask']  
    # 为了让模型能够处理不同长度的输入，通常会将输入填充到相同长度，但我们不希望模型关注这些填充的部分。
    # [CLS] 我 爱 自然 语言 处理 [SEP] [PAD] [PAD] [PAD]
    # attention_mask = [1, 1, 1, 1, 1, 1, 0, 0, 0]
}
train_dataset = Dataset.from_dict(data_dict)  

# 数据整理
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False  # mlm 表示是否进行掩码语言模型训练 (Masked Language Model, MLM)
    # 当 mlm=True 时，MLM 会随机掩盖部分输入 token，模型任务是预测被掩盖的 token，
    # 适用于 BERT 等模型。mlm=False 则不进行掩盖，通常适用于 GPT 等非掩码模型。
)

train_args = TrainingArguments(
    output_dir='./output',  # 训练输出的目录，存储模型的权重、配置文件等
    overwrite_output_dir=True,  # 是否覆盖之前的输出文件，如果设为 False，训练会在发现已有输出文件时报错
    num_train_epochs=5,  # 训练的轮次
    per_device_train_batch_size=32,  # 每个设备（如 GPU）的训练批次大小，这里是 32。较大的 batch size 可能会加速训练，但需要更多显存
    save_steps=3000,  # 每隔 3000 步保存模型
    save_total_limit=2,  # 只保存最新的 2 个模型检查点，较大的值会保留更多模型版本，但会占用更多存储空间
    warmup_steps=50,   # 训练开始时的学习率预热步数，在训练的初始阶段，学习率会从一个较小的值逐渐增加到预设的学习率。这种预热策略可以帮助模型在训练初期避免梯度爆炸，更稳定。
    logging_dir='./logs',  # 日志文件存放目录
    logging_steps=200,  # 每 200 步记录一次日志
    report_to=[]  # 指定将训练过程报告到哪里，比如 `report_to=['tensorboard']` 用于可视化    `report_to=["wandb"]` 将训练过程报告到 Weights & Biases (W&B) 平台
)

trainer = Trainer(
    model = model,
    args = train_args,
    data_collator = data_collator,
    train_dataset = train_dataset
)

In [ ]:
trainer.train()